# Interactive Plotting

In this notebook we discuss two ways to enable user interaction in visualisations:

1. with interactive widgets to modify static visualizations
2. with interactive visualizations


## Interactive Widgets

`ipywidget`s are interactive HTML widgets for Jupyter notebooks/JupyterLab.
Υou get HTML elements in the notebook with which data can be entered and changed.

Simple interaction is possible with IPython by default. That means whenever the user changes some parameter the visualization is recreated on the server-side and sent to the client. 

In [1]:
from ipywidgets import interact, interact_manual

In [2]:
@interact(text='Hello', slider=(0,10),check=True,categories=['red','green','blue'])
def react(text, slider,check,categories):
    print(text,slider*10,check,categories)

interactive(children=(Text(value='Hello', description='text'), IntSlider(value=5, description='slider', max=10…

In [3]:
@interact_manual(text='Hello', slider=(0,10),check=True,categories=['red','green','blue'])
def react(text, slider,check,categories):
    print(text,slider*10,check,categories)

interactive(children=(Text(value='Hello', description='text'), IntSlider(value=5, description='slider', max=10…

In [4]:
import pandas as pd
cars = pd.read_csv('data/mtcars.csv')

@interact(bins=(5, 25, 5),color=['red','green','orange','blue'])
def show_distplot(bins,color):
    cars['mpg'].hist(bins=bins, color=color)

interactive(children=(IntSlider(value=15, description='bins', max=25, min=5, step=5), Dropdown(description='co…

### Custom-built widgets

http://nbviewer.ipython.org/github/ipython/ipython/blob/3.x/examples/Interactive%20Widgets/Widget%20List.ipynb 

In [5]:
from ipywidgets import widgets

[widget for widget in dir(widgets) if not widget.endswith('Widget') and widget[0] == widget[0].upper() and widget[0] != '_']

['Accordion',
 'AppLayout',
 'Audio',
 'BoundedFloatText',
 'BoundedIntText',
 'Box',
 'Button',
 'ButtonStyle',
 'CallbackDispatcher',
 'Checkbox',
 'Color',
 'ColorPicker',
 'Combobox',
 'Controller',
 'DatePicker',
 'Datetime',
 'Dropdown',
 'FileUpload',
 'FloatLogSlider',
 'FloatProgress',
 'FloatRangeSlider',
 'FloatSlider',
 'FloatText',
 'GridBox',
 'GridspecLayout',
 'HBox',
 'HTML',
 'HTMLMath',
 'Image',
 'IntProgress',
 'IntRangeSlider',
 'IntSlider',
 'IntText',
 'Label',
 'Layout',
 'NumberFormat',
 'Output',
 'Password',
 'Play',
 'RadioButtons',
 'Select',
 'SelectMultiple',
 'SelectionRangeSlider',
 'SelectionSlider',
 'SliderStyle',
 'Style',
 'Tab',
 'Text',
 'Textarea',
 'ToggleButton',
 'ToggleButtons',
 'ToggleButtonsStyle',
 'TwoByTwoLayout',
 'VBox',
 'Valid',
 'Video']

In [6]:
@interact(bins=widgets.FloatText(value=5))
def show_distplot(bins):
    cars['mpg'].hist(bins=int(bins))

interactive(children=(FloatText(value=5.0, description='bins'), Output()), _dom_classes=('widget-interact',))

In [7]:
text_widget = widgets.Textarea(value='Hello', description='text area')
slider_widget = widgets.BoundedFloatText(5,min=0,max=10, description='slider area')
check_widget = widgets.Checkbox(True,description="CheckboxWidget")
toggle = widgets.RadioButtons(options=['red','green','blue'], description="RadioButtonsWidget")

@interact(text=text_widget, slider=slider_widget,check=check_widget,categories=toggle)
def react(text, slider,check,categories):
    print(text,slider*10,check,categories)

interactive(children=(Textarea(value='Hello', description='text area'), BoundedFloatText(value=5.0, descriptio…

In [8]:
b = widgets.Button(description="Update")
checkbox = widgets.Checkbox(description="CheckboxWidget")

tab1_children = [b,
                 checkbox,
                 widgets.Dropdown(options=['A','B'], description="DropdownWidget"),
                 widgets.RadioButtons(options=['A','B'], description="RadioButtonsWidget"),
                 widgets.Select(options=['A','B'], description="SelectWidget"),
                 widgets.Text(description="TextWidget"),
                 widgets.Textarea(description="TextareaWidget"),
                 widgets.ToggleButton(description="ToggleButtonWidget"),
                 widgets.ToggleButtons(options=["Value 1", "Value2"], description="ToggleButtonsWidget"),
                 ]

tab2_children = [widgets.BoundedFloatText(description="BoundedFloatTextWidget"),
                 widgets.BoundedIntText(description="BoundedIntTextWidget"),
                 widgets.FloatSlider(description="FloatSliderWidget"),
                 widgets.FloatText(description="FloatTextWidget"),
                 widgets.IntSlider(description="IntSliderWidget"),
                 widgets.IntText(description="IntTextWidget"),
                 ]

tab1 = widgets.Box(children=tab1_children)
tab2 = widgets.Box(children=tab2_children)


i = widgets.Accordion(children=[tab1, tab2])

i.set_title(0,"Basic Widgets")
i.set_title(1,"Numbers Input")

from IPython.display import display

def button_clicked(bb):
    print(checkbox.value)
    #TODO update plot

b.on_click(button_clicked)

display(i)

Accordion(children=(Box(children=(Button(description='Update', style=ButtonStyle()), Checkbox(value=False, des…

### Gapminder Example
Convert the static plot from before into an interactive one where you can slide through the year:

In [9]:
import matplotlib.pyplot as plt
import seaborn as sns

colors = sns.color_palette()

gap = pd.read_csv('data/gapminder-unfiltered.tsv',index_col=0, sep='\t')
pop_max = gap['pop'].max()

@interact(year=(gap.year.min(), gap.year.max()))
def plot_gapminder(year):
    gapyear = gap[gap.year == year]
    for (name, group),color in zip(gapyear.groupby('continent'),colors):
        plt.scatter(x=group['lifeExp'],y=group['gdpPercap'],label=name, c=[color],s=(group['pop']/pop_max)*400)
    plt.yscale('log')
    plt.title('Life Expectancy vs GDP')
    plt.xlabel('Life Expectancy')
    plt.ylabel('GDP Per Cap')
    plt.xlim(gap.gdpPercap.min(),gap.gdpPercap.max())
    plt.xlim(gap.lifeExp.min(),gap.lifeExp.max())
    plt.legend()

interactive(children=(IntSlider(value=1978, description='year', max=2007, min=1950), Output()), _dom_classes=(…

## Interactive Visualizations (with Altair)
[Altair](https://altair-viz.github.io) is *declarative*  visualization library for Python and based on [Vega-Lite](https://vega.github.io/vega-lite/).

Declartive means that - in Altair - you specify what you want the visualization to use, i.e.,
* Which attribute gets mapped to x and y.
* Color the marks by a one attribute and set vary its size based on another, etc.

So the key idea is that you link attributes to visual encoding channels.

Altair visualizations can be made interactive easily and allows you to link multiple visualizations.

### Intro to Altair
The first step to create an Altair visualization is to specify a [mark type](https://altair-viz.github.io/user_guide/marks.html).
Altair comes with `point`, `line`, `bar`, `area`, `geoshape`, and more.

Then you are able to [encode](https://altair-viz.github.io/user_guide/encoding.html) `position`, `color`, `shape`, and `size` based on your data.

In [10]:
import altair as alt

import pandas as pd
import numpy as np

In [11]:
alt.__version__

'4.1.0'

In [12]:
#use a standard dataset of heterogenous data
from vega_datasets import data
#cars = pd.read_csv('data/mtcars.csv')
cars = data.cars() # more cars + categorical columns
cars.tail()

,Name,Miles_per_Gallon,Cylinders,Displacement,Horsepower,Weight_in_lbs,Acceleration,Year,Origin
401,ford mustang gl,27.0,4,140.0,86.0,2790,15.6,1982-01-01,USA
402,vw pickup,44.0,4,97.0,52.0,2130,24.6,1982-01-01,Europe
403,dodge rampage,32.0,4,135.0,84.0,2295,11.6,1982-01-01,USA
404,ford ranger,28.0,4,120.0,79.0,2625,18.6,1982-01-01,USA
405,chevy s-10,31.0,4,119.0,82.0,2720,19.4,1982-01-01,USA


Let's start by taking another look at the visualizations we discussed in the last part. 

#### Scatterplot

In [13]:
alt.Chart(cars).mark_point() # 406 cars -> 406 points

alt.Chart(...)

In [14]:
alt.Chart(cars).mark_point().encode(x='Miles_per_Gallon')
#alt.Chart(cars).mark_point(opacity=0.1).encode(x='Miles_per_Gallon') # with opacity

alt.Chart(...)

In [15]:
alt.Chart(cars).mark_tick().encode(x='Miles_per_Gallon')

alt.Chart(...)

In [16]:
alt.Chart(cars).mark_point().encode(x='Miles_per_Gallon', color='Cylinders')

alt.Chart(...)

In [17]:
# We use a point as mark
alt.Chart(cars).mark_point().encode(
    x='Miles_per_Gallon',
    y='Horsepower',
    color='Cylinders' # O=Ordinal, N=Nominal, Q=Quantitative; see https://altair-viz.github.io/user_guide/encoding.html#encoding-data-types
)

alt.Chart(...)

In [18]:
# But some other marks work just as well (or almost 😉 ).
import ipywidgets as widgets

baseChart = alt.Chart(cars);

# https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20List.html#Selection-widgets
@widgets.interact(mark_type = widgets.Dropdown(
    options = [('point', baseChart.mark_point()), # key value pairs to pass a function to the function
                 ('circle', baseChart.mark_circle()), 
                 ('square', baseChart.mark_square()), 
                 ('tick', baseChart.mark_tick()), 
                 ('line', baseChart.mark_line())],
    description = 'Mark Type:'))
def show_plot(mark_type):
    # You have to return the chart to make it visible.
    return mark_type.encode(
    x='Miles_per_Gallon',
    y='Horsepower',
    color='Cylinders:N'
)   


interactive(children=(Dropdown(description='Mark Type:', options=(('point', alt.Chart(...)), ('circle', alt.Ch…

#### Histogram (aggregating Data)

In [19]:
# Histograms aggregate data, i.e. the data is put into bins, and the number of items in the bin is shown

alt.Chart(cars).mark_bar().encode(
    x='Horsepower'
    #alt.X("Horsepower", bin=True)
)
# --> 400 bars: see svg

alt.Chart(...)

In [20]:
alt.Chart(cars).mark_bar().encode(
    alt.X("Horsepower", bin=True),
    #alt.X("Horsepower", bin=alt.BinParams(maxbins=50)),
    y='count()'
)

alt.Chart(...)

#### Stacked Histogram

In [21]:
alt.Chart(cars).mark_bar().encode(
    x = alt.X("Horsepower", bin=True),
    y='count()',
    color='Origin'
)

#Normalized:
# alt.Chart(cars).mark_bar().encode(
#     x = alt.X("Horsepower", bin=True),
#     y=alt.Y('count()', stack='normalize'),
#     color='Origin'
# )

alt.Chart(...)

#### Grouped Histogram

In [22]:
alt.Chart(cars).mark_bar().encode(
    x = alt.X("Horsepower", bin=True),
    y='count()',
    column='Origin'
)

# #Flip x & column
# alt.Chart(cars).mark_bar().encode(
#     column = alt.X("Horsepower", bin=True),
#     y='count()',
#     x ='Origin'
# )

# Pretty axes
# column is an encoding, no axis (even if it looks like one) --> change header not axis property
# alt.Chart(cars).mark_bar().encode(
#     column = alt.Column(
#         "Horsepower",
#         bin=True,
#         header=alt.Header(titleOrient='bottom', title="Horsepower", labelOrient='bottom'),
#         spacing=0),
#     y = alt.Y('count()'),
#     x = alt.X('Origin', axis=None),    
#     color = 'Origin'
# ).configure_view(
#     strokeWidth=0.0 # remove facet borders
# )


alt.Chart(...)

#### Faceting

In [23]:
alt.Chart(cars).mark_point().encode(
    #opacity=alt.value(0.5) # a constant value
    x='Miles_per_Gallon',
    y='Horsepower',
    color='Cylinders:N',
    column='Cylinders'
).properties(
    width=150, # per plot; see https://altair-viz.github.io/user_guide/configuration.html#view-configuration
    height=150
)

alt.Chart(...)

In [24]:
#Or call .facet()
# https://altair-viz.github.io/user_guide/compound_charts.html#faceted-charts

alt.Chart(cars).mark_point().encode(
    x='Miles_per_Gallon',
    y='Horsepower',
    color='Cylinders:N'
) \
.properties(width=150, height=150) \
.facet('Cylinders') # set properties before faceting

alt.FacetChart(...)

#### Tooltips

In [25]:
# We use a point as mark
alt.Chart(cars).mark_point().encode(
    x='Miles_per_Gallon',
    y='Horsepower',
    color='Cylinders:N',
    tooltip=['Name', 'Origin']
)

alt.Chart(...)

#### Gapminder Example

In [26]:
gap = pd.read_csv('data/gapminder-unfiltered.tsv', sep='\t')
gap2007 = gap[gap.year == 2007]

In [27]:
alt.Chart(gap2007).mark_circle().encode(
    x=alt.X('gdpPercap',  scale=alt.Scale(type='log')),
    y='lifeExp',
    #y=alt.Y('lifeExp', scale=alt.Scale(zero=False)),
    color='continent',
    size='pop',
    tooltip='country'
)

alt.Chart(...)

### Adding Interactivity
#### Zoom & Pan

In [28]:
vis = alt.Chart(gap2007).mark_circle().encode(
    x=alt.X('gdpPercap',  scale=alt.Scale(type='log')),
    y=alt.Y('lifeExp', scale=alt.Scale(zero=False)),
    color='continent',
    size='pop',
    tooltip='country'
)

# Simple
vis.interactive()

# Verbose
#selection = alt.selection_interval(bind='scales') # selection of type "interval", bound to the scales
#vis.add_selection(
#    selection # bind this selection to our chart
#)

alt.Chart(...)

#### Brushing

In [29]:
brush = alt.selection_interval()  # selection of type "interval"

alt.Chart(cars).mark_point().encode(
    x='Miles_per_Gallon',
    y='Horsepower',
    color=alt.condition(brush, 'Cylinders:N', alt.value('lightgray')) # unselected become lightgrey if condition(brushed)=false
).add_selection(
    brush  # bind this brush to our chart
)

alt.Chart(...)

#### Linking

In [30]:
brush = alt.selection_interval(encodings=['x']) # Only seelct on x-axis

chart = alt.Chart(cars).mark_point().encode(
    y='Horsepower',
    color=alt.condition(brush, 'Cylinders:N', alt.value('lightgray'))
).add_selection(
    brush
)

# Link horizontal
chart.encode(x='Miles_per_Gallon') | chart.encode(x='Weight_in_lbs') #two charts combined with a vertical bar '|'

alt.HConcatChart(...)

In [31]:
# Link vertical
chart.encode(x='Miles_per_Gallon') & chart.encode(x='Weight_in_lbs')

alt.VConcatChart(...)

In [32]:
# Layered -> cant have two selections -> remove one
alt.Chart(cars).mark_point().encode(
    x='Miles_per_Gallon',
    y='Horsepower'
) + alt.Chart(cars).mark_point().encode(
    x='Displacement',
    y='Horsepower'
).add_selection(brush)

alt.LayerChart(...)

## Next

[Machine Learning using Scikit Learn](05_MachineLearning.ipynb)